In [4]:
import cv2
import numpy as np

In [99]:
image = cv2.imread("img/Tada Koe Hitotsu 4.png")
lines_image = image.copy()
height, width, channels = image.shape

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 10))
dilate = cv2.dilate(thresh, kernel, iterations=1)

In [100]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x : cv2.boundingRect(x)[1]) # Sort based on vertical order

staff_count = 0

for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    
    if w > width*0.8:
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)

        roi = lines_image[y:y+h, x:x+w]
        cv2.imwrite(f"res/roi_{staff_count}.jpg", roi)
        staff_count += 1

show_wait_destroy("dilate", dilate)   
show_wait_destroy("final", image)   

In [35]:
cv2.imwrite("res/dilate.jpg", dilate)
cv2.imwrite("res/final.jpg", image)

True

In [6]:
def show_wait_destroy(winname, img):
    cv2.imshow(winname, img)
    cv2.moveWindow(winname, 500, 0)
    cv2.waitKey(0)
    cv2.destroyWindow(winname)

In [23]:
# Extract horizontal staff lines
for i in range(5):
    roi = cv2.imread(f"res/roi_{i}.jpg")
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Show binary image
    show_wait_destroy("binary", bw)
    
    # Create the images that will use to extract the horizontal and vertical lines
    horizontal = np.copy(bw)
    width = horizontal.shape[1]
    
    # Specify size on horizontal axis
    cols = horizontal.shape[1]
    horizontal_size = cols // 30
    
    # Create structure element for extracting horizontal lines through morphology operations
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (round(width*0.8), 1))
    
    # Apply morphology operations
    horizontal = cv2.erode(horizontal, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure)
    
    # Filter based on thickness
    contours, _ = cv2.findContours(horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        # Get bounding box of each contour
        x, y, w, h = cv2.boundingRect(contour)
    
        # Filter based on height (thickness)
        if w < width*0.8 and h > 4:  # Adjust this value to filter out thicker beams
            cv2.drawContours(horizontal, [contour], -1, 0, -1)

    # Optional: Preserve edges
    # horizontal = cv2.dilate(horizontal, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)), iterations=1)
            
    # Show extracted horizontal lines
    show_wait_destroy("horizontal", horizontal)

cv2.imwrite("res/horizontal.jpg", horizontal)

True

In [6]:
def merge_intersecting_bounding_boxes(contours):
    # Step 1: Calculate initial bounding boxes
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

    # Step 2: Merge bounding boxes iteratively
    def boxes_intersect(box1, box2, min_overlap=2):
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        return x1 + w1 + min_overlap >= x2 and x2 + w2 + min_overlap >= x1 and y1 + h1 + min_overlap >= y2 and y2 + h2 + min_overlap >= y1

    def merge_boxes(box1, box2):
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        x_min = min(x1, x2)
        y_min = min(y1, y2)
        x_max = max(x1 + w1, x2 + w2)
        y_max = max(y1 + h1, y2 + h2)
        return (x_min, y_min, x_max - x_min, y_max - y_min)

    merged = True
    while merged:
        merged = False
        new_boxes = []
        skip_indices = set()

        # Compare each box with every other box
        for i, box1 in enumerate(bounding_boxes):
            if i in skip_indices:
                continue
            merged_box = box1
            for j, box2 in enumerate(bounding_boxes):
                if i != j and j not in skip_indices and boxes_intersect(merged_box, box2):
                    merged_box = merge_boxes(merged_box, box2)
                    skip_indices.add(j)
                    merged = True

            new_boxes.append(merged_box)
            skip_indices.add(i)

        bounding_boxes = new_boxes  # Update with the newly merged boxes

    return bounding_boxes

In [ ]:
# Remove the staff lines

for i in range(staff_count):
    roi = cv2.imread(f"res/roi_{i}.jpg")
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    bw = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Show binary image
    show_wait_destroy("binary", bw)
    vertical = np.copy(bw)
    
    # rows = vertical.shape[0]
    # vertical_size = max(rows // 80, 3)
    
    # Create structure element for extracting vertical lines through morphology operations
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4))
    
    # Apply morphology operations
    vertical = cv2.erode(vertical, verticalStructure)
    vertical = cv2.dilate(vertical, verticalStructure)

    # Reconstruct noteheads and beams
    notehead_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    reconstructed = cv2.dilate(vertical, notehead_kernel, iterations=1)

    # Invert the image to origin
    inverted = cv2.bitwise_not(reconstructed)

    # This is optional since we will use YOLO to do object detection
    # contours, _ = cv2.findContours(reconstructed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours = merge_intersecting_bounding_boxes(contours)
    # for x, y, w, h in contours:
    #     cv2.rectangle(reconstructed, (x, y), (x+w, y+h), (255, 0, 0), 1)
    
    # Show extracted vertical lines
    show_wait_destroy(f"res/roi_{i}_reconstructed.jpg", inverted)
    cv2.imwrite(f"res/roi_{i}_reconstructed.jpg", inverted)

In [ ]:
# Morphological method 2
# Remove the staff lines

for i in range(5):
    roi = cv2.imread(f"res/roi_{i}.jpg")
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    binary_image = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Show binary image
    show_wait_destroy("binary", binary_image)
        
    kernel_length = roi.shape[1] // 40  # Adjust based on the sheet size
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    detected_lines = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)

    # Step 4: Create a mask for the staff lines
    staff_line_mask = detected_lines.copy()

    # Step 5: Subtract staff lines from the binary image
    lines_removed = cv2.subtract(binary_image, staff_line_mask)

    # Step 6: Reconstruct noteheads and symbols
    notehead_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  # Small elliptical kernel
    reconstructed_image = cv2.dilate(lines_removed, notehead_kernel, iterations=1)
    
    # Show extracted vertical lines
    show_wait_destroy("vertical", reconstructed_image)
    

In [30]:
# Load the image in grayscale
roi = cv2.imread(f"res/roi_1.jpg")
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
bw = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)[1]

# Show the results
cv2.imshow("Original", gray)
cv2.imshow("Enhanced (Black, Shadows, Contrast)", bw)
cv2.waitKey(0)
cv2.destroyAllWindows()